In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input, Lambda
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import librosa

# Load and preprocess data
data = pd.read_csv('data.csv')
labels = pd.read_csv('label.csv')

X = data['ACC X'].values
y = labels.values
X = X.reshape(-1, 2500)  # Reshape for spectrogram computation

# Convert accelerometer data to spectrograms
def compute_spectrogram(data):
    spec = librosa.feature.melspectrogram(y=data, sr=1000)  # Adjust sr based on your data
    return spec

X_spectrograms = np.array([compute_spectrogram(x) for x in X])

# Convert labels to one-hot encoding
y_one_hot = tf.keras.utils.to_categorical(y, num_classes=55)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_spectrograms, y_one_hot, test_size=0.2, random_state=42)

# Build Siamese network
input_shape = X_spectrograms.shape[1:]  # Adjust the shape based on your spectrogram dimensions

input_left = Input(shape=input_shape)
input_right = Input(shape=input_shape)

base_network = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=input_shape),
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(256, activation='relu')
])

encoded_left = base_network(input_left)
encoded_right = base_network(input_right)

# Define contrastive loss function
def contrastive_loss(y_true, y_pred):
    margin = 1.0
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

# Lambda layer to calculate distances between the encoded representations
distance_layer = Lambda(lambda tensors: K.sqrt(K.sum(K.square(tensors[0] - tensors[1]), axis=-1, keepdims=True)))
distance = distance_layer([encoded_left, encoded_right])

# Build the Siamese model
siamese_model = Model(inputs=[input_left, input_right], outputs=distance)
siamese_model.compile(optimizer='adam', loss=contrastive_loss)


# Create a new model for fine-tuning
fine_tuning_model = Sequential([
    base_network,
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(55, activation='softmax')
])

fine_tuning_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the fine-tuning model
batch_size = 32  # Define an appropriate batch size
fine_tuning_epochs = 50
fine_tuning_history = fine_tuning_model.fit(X_train, y_train, epochs=fine_tuning_epochs, batch_size=batch_size, validation_data=(X_val, y_val))

# Save printouts to text files
with open('contrastive_learning_printout.txt', 'w') as f:
    f.write(str(fine_tuning_history.history))


Epoch 1/50
2/2 [==============================] - 3s 501ms/step - loss: 1115.9274 - accuracy: 0.0682 - val_loss: 115.2850 - val_accuracy: 0.4545
Epoch 2/50
2/2 [==============================] - 0s 123ms/step - loss: 370.8369 - accuracy: 0.4545 - val_loss: 275.5882 - val_accuracy: 0.4545
Epoch 3/50
2/2 [==============================] - 0s 126ms/step - loss: 421.7155 - accuracy: 0.4545 - val_loss: 147.9352 - val_accuracy: 0.3636
Epoch 4/50
2/2 [==============================] - 0s 117ms/step - loss: 269.2155 - accuracy: 0.4545 - val_loss: 93.0440 - val_accuracy: 0.3636
Epoch 5/50
2/2 [==============================] - 0s 112ms/step - loss: 300.2971 - accuracy: 0.2727 - val_loss: 78.8139 - val_accuracy: 0.3636
Epoch 6/50
2/2 [==============================] - 0s 111ms/step - loss: 359.3009 - accuracy: 0.3409 - val_loss: 129.5981 - val_accuracy: 0.4545
Epoch 7/50
2/2 [==============================] - 0s 123ms/step - loss: 189.0670 - accuracy: 0.3864 - val_loss: 224.8332 - val_accuracy: 